In [60]:
import numpy as np
import pandas as pd
import os
from IPython.display import display
import warnings
import ipynbname
from pathlib import Path
warnings.filterwarnings('ignore')

# 设置显示选项
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 100)

In [61]:
# direct_llm_id = "direct_llm_20samples_rag_evaluation_results_20251121_104026"
direct_llm_id = "direct_llm_120_samplesrag_evaluation_results_20251121_104114"
directory = Path("../results/aip_saved/")
direct_llm_baseline_path = list(directory.glob(f"{direct_llm_id}*.csv"))
direct_llm_baseline_df = pd.read_csv(direct_llm_baseline_path[0])

# crud_id = "crud_hybrid_aip_test_20_full_20251121_161550"
crud_id = "crud_hybrid_aip_test_full_20251121_161550"
crud_baseline_path = list(directory.glob(f"{crud_id}*.csv"))
crud_baseline_df = pd.read_csv(crud_baseline_path[0])

# 获取所有实验文件夹
# results_base_dir = "../results/aip_saved/"
# experiment_name = "pre_search/original"
results_base_dir = '../results/'
experiment_name = '8b_short'
experiment_root = os.path.join(results_base_dir, experiment_name)
experiment_folders = []
for experiment_id in os.listdir(experiment_root):
    if os.path.isdir(os.path.join(experiment_root, experiment_id)):
        experiment_folders.append(os.path.join(experiment_root, experiment_id))
print(f"\n处理{experiment_root}中的{len(experiment_folders)}个实验文件夹")


处理../results/8b_short中的2个实验文件夹


In [62]:
import scipy.stats as stats

# 定义需要分析的数值指标列
metric_columns = [
    'overall_score', 'metric_with_token', 'num_search_calls', 'perf_score', 'time_estimate_sec', 'llm_judge_score', 'rejection_recall', 
    'ndcg', 'ncg', 'mrr', 'ndcg_at_3', 'ndcg_at_5',
    "pre_search_ndcg", "pre_search_mrr", "pre_search_ncg",
    'intermediate_ndcg', 'intermediate_ncg', 'intermediate_ndcg_at_3', 'intermediate_ndcg_at_5',
    'ndcg_inter-final_diff', 'ncg_inter-final_diff',
    'search_agent_ndcg', 'search_agent_ncg',
    'ndcg_search_agent-final_diff', 'ncg_search_agent-final_diff',
    'ndcg_inter-search_agent_diff', 'ncg_inter-search_agent_diff',
    'execution_time_seconds'
]

# 读取所有实验的CSV文件
all_experiments_data = []
for experiment_folder in experiment_folders:
    csv_files = [f for f in os.listdir(experiment_folder) 
                    if f.endswith('.csv') and 'rag_evaluation' in f]
    if csv_files:
        csv_path = os.path.join(experiment_folder, csv_files[0])
        df = pd.read_csv(csv_path)
        df['experiment_id'] = os.path.basename(experiment_folder)
        all_experiments_data.append(df)
        print(experiment_folder.split('/')[-1], df['overall_score'].mean())

# 合并所有实验数据
combined_df = pd.concat(all_experiments_data, ignore_index=True)
print(f"总共加载了 {len(all_experiments_data)} 个实验，包含 {len(combined_df)} 条记录")
print(f"每个实验的样本数: {combined_df.groupby('experiment_id').size().values}")
# Group by 'question_id' and select the maximum 'overall_score' for each sample
# Select rows with the maximum overall_score for each question_id
max_overall_scores_df = combined_df.loc[combined_df.groupby('question_id')['overall_score'].idxmax()]

# Save the result to a new CSV file
max_root = os.path.join(experiment_root, 'max')
os.makedirs(max_root, exist_ok=True)
max_overall_scores_all_columns_output_path = os.path.join(max_root, "rag_evaluation.csv")
max_overall_scores_df.to_csv(max_overall_scores_all_columns_output_path, index=False)

print(f"\n✓ 已保存每个样本的最大 overall_score（包含所有列）到: {max_overall_scores_all_columns_output_path}")

experiment_20251206_173252 0.6539520006902522
experiment_20251206_202919 0.6302324716426956
总共加载了 2 个实验，包含 236 条记录
每个实验的样本数: [118 118]

✓ 已保存每个样本的最大 overall_score（包含所有列）到: ../results/8b_short/max/rag_evaluation.csv


In [ ]:
# 计算每个sample的各个数值指标的均值（跨所有实验）
sample_means = combined_df.groupby('question_id')[metric_columns].mean()
for sample in sample_means.index:
    sample_means.at[sample, 'question_type'] = combined_df[combined_df['question_id'] == sample]['question_type'].iloc[0]
# 按overall_score排序
sample_means_sorted = sample_means.sort_values('overall_score', ascending=False)

print("=" * 100)
print("每个样本的各指标均值（按overall_score降序排列）")
print("=" * 100)
display(sample_means_sorted)

# 保存每个样本的均值到CSV
sample_means_output_path = os.path.join(experiment_root, "rag_evaluation_sample_means_across_experiments.csv")
sample_means_sorted.to_csv(sample_means_output_path)
print(f"\n✓ 已保存每个样本的均值到: {sample_means_output_path}")


KeyError: "Columns not found: 'time_estimate_sec'"

: 

In [ ]:
# 计算每个sample的overall_score的置信区间
def calculate_confidence_interval(data, confidence=0.95):
    """
    计算置信区间
    
    Args:
        data: 数据数组
        confidence: 置信水平（默认95%）
    
    Returns:
        mean, lower_bound, upper_bound, std, sem
    """
    n = len(data)
    if n == 0:
        return np.nan, np.nan, np.nan, np.nan, np.nan
    
    mean = np.mean(data)
    std = np.std(data, ddof=1)  # 样本标准差
    sem = stats.sem(data)  # 标准误
    
    # 使用t分布计算置信区间
    confidence_interval = stats.t.interval(confidence, n-1, loc=mean, scale=sem)
    
    return mean, confidence_interval[0], confidence_interval[1], std, sem

# 计算每个sample的overall_score统计信息
sample_overall_stats = []
for question_id in combined_df['question_id'].unique():
    sample_data = combined_df[combined_df['question_id'] == question_id]['overall_score'].dropna()
    mean, ci_lower, ci_upper, std, sem = calculate_confidence_interval(sample_data.values)
    
    sample_overall_stats.append({
        'question_id': question_id,
        'overall_score_mean': mean,
        'overall_score_ci_lower': ci_lower,
        'overall_score_ci_upper': ci_upper,
        'overall_score_std': std,
        'overall_score_sem': sem,
        'ci_width': ci_upper - ci_lower,
        'n_experiments': len(sample_data)
    })

sample_overall_stats_df = pd.DataFrame(sample_overall_stats)
sample_overall_stats_df = sample_overall_stats_df.sort_values('overall_score_mean', ascending=False)

print("=" * 100)
print("每个样本的overall_score置信区间分析（按均值降序排列）")
print("=" * 100)
display(sample_overall_stats_df)

# 保存到CSV
sample_ci_output_path = os.path.join(experiment_root, "sample_overall_score_confidence_intervals.csv")
sample_overall_stats_df.to_csv(sample_ci_output_path, index=False)
print(f"\n✓ 已保存每个样本的overall_score置信区间到: {sample_ci_output_path}")


每个样本的overall_score置信区间分析（按均值降序排列）


,question_id,overall_score_mean,overall_score_ci_lower,overall_score_ci_upper,overall_score_std,overall_score_sem,ci_width,n_experiments
117,refusal_sample_0020,1.000000,NaN,NaN,NaN,NaN,NaN,1
0,crud_sample_0001,1.000000,NaN,NaN,NaN,NaN,NaN,1
1,crud_sample_0002,1.000000,NaN,NaN,NaN,NaN,NaN,1
116,refusal_sample_0019,1.000000,NaN,NaN,NaN,NaN,NaN,1
100,refusal_sample_0003,1.000000,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...
59,crud_sample_0060,0.334639,NaN,NaN,NaN,NaN,NaN,1
47,crud_sample_0048,0.306574,NaN,NaN,NaN,NaN,NaN,1
80,crud_sample_0082,0.217320,NaN,NaN,NaN,NaN,NaN,1
86,crud_sample_0089,0.100000,NaN,NaN,NaN,NaN,NaN,1



✓ 已保存每个样本的overall_score置信区间到: ../results/aip_saved/pre_search/original/sample_overall_score_confidence_intervals.csv


In [ ]:
# 计算所有指标的全局统计分析
def calculate_metric_statistics(df, metric_columns, confidence=0.95):
    """
    计算所有数值指标的统计信息
    
    Args:
        df: 包含所有实验数据的DataFrame
        metric_columns: 需要分析的指标列名列表
        confidence: 置信水平
    
    Returns:
        统计信息的DataFrame
    """
    stats_list = []
    
    for metric in metric_columns:
        if metric not in df.columns:
            continue
            
        data = df[metric].dropna().values
        
        if len(data) == 0:
            continue
        
        mean, ci_lower, ci_upper, std, sem = calculate_confidence_interval(data, confidence)
        
        stats_list.append({
            'metric': metric,
            'mean': mean,
            'ci_lower': ci_lower,
            'ci_upper': ci_upper,
            'ci_width': ci_upper - ci_lower,
            'std': std,
            'sem': sem,
            'min': np.min(data),
            'q25': np.percentile(data, 25),
            'median': np.median(data),
            'q75': np.percentile(data, 75),
            'max': np.max(data),
            'n_samples': len(data),
            'n_missing': df[metric].isna().sum()
        })
    
    return pd.DataFrame(stats_list)

# 计算全局统计信息
global_stats_df = calculate_metric_statistics(combined_df, metric_columns)

print("=" * 100)
print("所有指标的全局统计分析（包含均值、置信区间、分位数等）")
print("=" * 100)
display(global_stats_df)

# 保存到CSV
global_stats_output_path = os.path.join(experiment_root, "global_metrics_statistics.csv")
global_stats_df.to_csv(global_stats_output_path, index=False)
print(f"\n✓ 已保存全局统计信息到: {global_stats_output_path}")


所有指标的全局统计分析（包含均值、置信区间、分位数等）


,metric,mean,ci_lower,ci_upper,ci_width,std,sem,min,q25,median,q75,max,n_samples,n_missing
0,overall_score,7.811634e-01,7.368115e-01,8.255153e-01,8.870373e-02,2.432705e-01,2.239487e-02,0.100000,0.600000,0.850000,1.000000,1.000000e+00,118,0
1,metric_with_token,6.575681e-01,6.254513e-01,6.896848e-01,6.423356e-02,1.761610e-01,1.621693e-02,0.162326,0.522084,0.705086,0.817683,8.233771e-01,118,0
2,num_search_calls,2.000000e+00,NaN,NaN,NaN,0.000000e+00,0.000000e+00,2.000000,2.000000,2.000000,2.000000,2.000000e+00,118,0
3,perf_score,3.485797e-01,3.453778e-01,3.517815e-01,6.403717e-03,1.756224e-02,1.616735e-03,0.287060,0.336130,0.350500,0.364000,3.818200e-01,118,0
4,time_estimate_sec,6.427288e+00,6.027067e+00,6.827509e+00,8.004416e-01,2.195216e+00,2.020860e-01,2.270000,4.500000,6.190000,7.985000,1.412000e+01,118,0
5,llm_judge_score,6.020408e-01,5.248485e-01,6.792332e-01,1.543847e-01,3.850237e-01,3.889326e-02,0.000000,0.200000,0.700000,1.000000,1.000000e+00,98,20
6,ndcg,8.709650e-01,8.241834e-01,9.177465e-01,9.356316e-02,2.333393e-01,2.357083e-02,0.000000,0.765361,1.000000,1.000000,1.000000e+00,98,20
7,ncg,8.503401e-01,7.990010e-01,9.016792e-01,1.026782e-01,2.560715e-01,2.586713e-02,0.000000,0.666667,1.000000,1.000000,1.000000e+00,98,20
8,mrr,9.523810e-01,9.131072e-01,9.916547e-01,7.854747e-02,1.958914e-01,1.978802e-02,0.000000,1.000000,1.000000,1.000000,1.000000e+00,98,20
9,ndcg_at_3,8.709650e-01,8.241834e-01,9.177465e-01,9.356316e-02,2.333393e-01,2.357083e-02,0.000000,0.765361,1.000000,1.000000,1.000000e+00,98,20



✓ 已保存全局统计信息到: ../results/aip_saved/pre_search/original/global_metrics_statistics.csv


In [ ]:
# 计算每个指标在每个样本上的置信区间
def calculate_all_metrics_ci_per_sample(df, metric_columns, confidence=0.95):
    """
    计算每个样本在所有指标上的置信区间
    
    Args:
        df: 包含所有实验数据的DataFrame
        metric_columns: 需要分析的指标列名列表
        confidence: 置信水平
    
    Returns:
        包含所有样本所有指标置信区间的DataFrame
    """
    results = []
    
    for question_id in df['question_id'].unique():
        sample_data = df[df['question_id'] == question_id]
        row = {'question_id': question_id}
        
        for metric in metric_columns:
            if metric not in sample_data.columns:
                continue
                
            data = sample_data[metric].dropna().values
            
            if len(data) == 0:
                row[f'{metric}_mean'] = np.nan
                row[f'{metric}_ci_lower'] = np.nan
                row[f'{metric}_ci_upper'] = np.nan
                row[f'{metric}_std'] = np.nan
            else:
                mean, ci_lower, ci_upper, std, sem = calculate_confidence_interval(data, confidence)
                row[f'{metric}_mean'] = mean
                row[f'{metric}_ci_lower'] = ci_lower
                row[f'{metric}_ci_upper'] = ci_upper
                row[f'{metric}_std'] = std
        
        results.append(row)
    
    return pd.DataFrame(results)

# 计算所有指标的样本级置信区间
all_metrics_ci_df = calculate_all_metrics_ci_per_sample(combined_df, metric_columns)
all_metrics_ci_df = all_metrics_ci_df.sort_values('overall_score_mean', ascending=False)

print("=" * 100)
print("每个样本的所有指标置信区间（按overall_score_mean降序排列）")
print("=" * 100)
print(f"数据维度: {all_metrics_ci_df.shape}")
display(all_metrics_ci_df)

# 保存到CSV
all_metrics_ci_output_path = os.path.join(experiment_root, "all_metrics_confidence_intervals_per_sample.csv")
all_metrics_ci_df.to_csv(all_metrics_ci_output_path, index=False)
print(f"\n✓ 已保存所有样本的所有指标置信区间到: {all_metrics_ci_output_path}")


每个样本的所有指标置信区间（按overall_score_mean降序排列）
数据维度: (118, 113)


,question_id,overall_score_mean,overall_score_ci_lower,overall_score_ci_upper,overall_score_std,metric_with_token_mean,metric_with_token_ci_lower,metric_with_token_ci_upper,metric_with_token_std,num_search_calls_mean,num_search_calls_ci_lower,num_search_calls_ci_upper,num_search_calls_std,perf_score_mean,perf_score_ci_lower,perf_score_ci_upper,perf_score_std,time_estimate_sec_mean,time_estimate_sec_ci_lower,time_estimate_sec_ci_upper,time_estimate_sec_std,llm_judge_score_mean,llm_judge_score_ci_lower,llm_judge_score_ci_upper,llm_judge_score_std,rejection_recall_mean,rejection_recall_ci_lower,rejection_recall_ci_upper,rejection_recall_std,ndcg_mean,ndcg_ci_lower,ndcg_ci_upper,ndcg_std,ncg_mean,ncg_ci_lower,ncg_ci_upper,ncg_std,mrr_mean,mrr_ci_lower,mrr_ci_upper,mrr_std,ndcg_at_3_mean,ndcg_at_3_ci_lower,ndcg_at_3_ci_upper,ndcg_at_3_std,ndcg_at_5_mean,ndcg_at_5_ci_lower,ndcg_at_5_ci_upper,ndcg_at_5_std,pre_search_ndcg_mean,pre_search_ndcg_ci_lower,pre_search_ndcg_ci_upper,pre_search_ndcg_std,pre_search_mrr_mean,pre_search_mrr_ci_lower,pre_search_mrr_ci_upper,pre_search_mrr_std,pre_search_ncg_mean,pre_search_ncg_ci_lower,pre_search_ncg_ci_upper,pre_search_ncg_std,intermediate_ndcg_mean,intermediate_ndcg_ci_lower,intermediate_ndcg_ci_upper,intermediate_ndcg_std,intermediate_ncg_mean,intermediate_ncg_ci_lower,intermediate_ncg_ci_upper,intermediate_ncg_std,intermediate_ndcg_at_3_mean,intermediate_ndcg_at_3_ci_lower,intermediate_ndcg_at_3_ci_upper,intermediate_ndcg_at_3_std,intermediate_ndcg_at_5_mean,intermediate_ndcg_at_5_ci_lower,intermediate_ndcg_at_5_ci_upper,intermediate_ndcg_at_5_std,ndcg_inter-final_diff_mean,ndcg_inter-final_diff_ci_lower,ndcg_inter-final_diff_ci_upper,ndcg_inter-final_diff_std,ncg_inter-final_diff_mean,ncg_inter-final_diff_ci_lower,ncg_inter-final_diff_ci_upper,ncg_inter-final_diff_std,search_agent_ndcg_mean,search_agent_ndcg_ci_lower,search_agent_ndcg_ci_upper,search_agent_ndcg_std,search_agent_ncg_mean,search_agent_ncg_ci_lower,search_agent_ncg_ci_upper,search_agent_ncg_std,ndcg_search_agent-final_diff_mean,ndcg_search_agent-final_diff_ci_lower,ndcg_search_agent-final_diff_ci_upper,ndcg_search_agent-final_diff_std,ncg_search_agent-final_diff_mean,ncg_search_agent-final_diff_ci_lower,ncg_search_agent-final_diff_ci_upper,ncg_search_agent-final_diff_std,ndcg_inter-search_agent_diff_mean,ndcg_inter-search_agent_diff_ci_lower,ndcg_inter-search_agent_diff_ci_upper,ndcg_inter-search_agent_diff_std,ncg_inter-search_agent_diff_mean,ncg_inter-search_agent_diff_ci_lower,ncg_inter-search_agent_diff_ci_upper,ncg_inter-search_agent_diff_std,execution_time_seconds_mean,execution_time_seconds_ci_lower,execution_time_seconds_ci_upper,execution_time_seconds_std
117,refusal_sample_0020,1.000000,NaN,NaN,NaN,0.823377,NaN,NaN,NaN,2.0,NaN,NaN,NaN,0.38182,NaN,NaN,NaN,2.27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.172,NaN,NaN,NaN
0,crud_sample_0001,1.000000,NaN,NaN,NaN,0.818531,NaN,NaN,NaN,2.0,NaN,NaN,NaN,0.36486,NaN,NaN,NaN,4.39,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,1.00,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,0.000000e+00,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.112,NaN,NaN,NaN
1,crud_sample_0002,1.000000,NaN,NaN,NaN,0.819091,NaN,NaN,NaN,2.0,NaN,NaN,NaN,0.36682,NaN,NaN,NaN,4.15,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,1.000000,NaN,NaN,


✓ 已保存所有样本的所有指标置信区间到: ../results/aip_saved/pre_search/original/all_metrics_confidence_intervals_per_sample.csv


In [ ]:
# 汇总统计报告
print("\n" + "=" * 100)
print("置信区间分析汇总报告")
print("=" * 100)

print(f"\n实验配置:")
print(f"  - 实验名称: {experiment_name}")
print(f"  - 实验数量: {len(experiment_folders)}")
print(f"  - 样本总数: {len(combined_df['question_id'].unique())}")
print(f"  - 总记录数: {len(combined_df)}")
print(f"  - 置信水平: 95%")

print(f"\n关键指标统计 (overall_score):")
overall_global = global_stats_df[global_stats_df['metric'] == 'overall_score'].iloc[0]
print(f"  - 均值: {overall_global['mean']:.4f}")
print(f"  - 95% 置信区间: [{overall_global['ci_lower']:.4f}, {overall_global['ci_upper']:.4f}]")
print(f"  - 标准差: {overall_global['std']:.4f}")
print(f"  - 中位数: {overall_global['median']:.4f}")
print(f"  - 范围: [{overall_global['min']:.4f}, {overall_global['max']:.4f}]")

print(f"\n关键指标统计 (metric_with_token):")
overall_global = global_stats_df[global_stats_df['metric'] == 'metric_with_token'].iloc[0]
print(f"  - 均值: {overall_global['mean']:.4f}")
print(f"  - 95% 置信区间: [{overall_global['ci_lower']:.4f}, {overall_global['ci_upper']:.4f}]")
print(f"  - 标准差: {overall_global['std']:.4f}")
print(f"  - 中位数: {overall_global['median']:.4f}")
print(f"  - 范围: [{overall_global['min']:.4f}, {overall_global['max']:.4f}]")

print(f"\n已生成的文件:")
print(f"  1. 每个样本的各指标均值: {sample_means_output_path}")
print(f"  2. 每个样本的overall_score置信区间: {sample_ci_output_path}")
print(f"  3. 全局指标统计信息: {global_stats_output_path}")
print(f"  4. 每个样本的所有指标置信区间: {all_metrics_ci_output_path}")

print("\n" + "=" * 100)



置信区间分析汇总报告

实验配置:
  - 实验名称: pre_search/original
  - 实验数量: 1
  - 样本总数: 118
  - 总记录数: 118
  - 置信水平: 95%

关键指标统计 (overall_score):
  - 均值: 0.7812
  - 95% 置信区间: [0.7368, 0.8255]
  - 标准差: 0.2433
  - 中位数: 0.8500
  - 范围: [0.1000, 1.0000]

关键指标统计 (metric_with_token):
  - 均值: 0.6576
  - 95% 置信区间: [0.6255, 0.6897]
  - 标准差: 0.1762
  - 中位数: 0.7051
  - 范围: [0.1623, 0.8234]

已生成的文件:
  1. 每个样本的各指标均值: ../results/aip_saved/pre_search/original/rag_evaluation_sample_means_across_experiments.csv
  2. 每个样本的overall_score置信区间: ../results/aip_saved/pre_search/original/sample_overall_score_confidence_intervals.csv
  3. 全局指标统计信息: ../results/aip_saved/pre_search/original/global_metrics_statistics.csv
  4. 每个样本的所有指标置信区间: ../results/aip_saved/pre_search/original/all_metrics_confidence_intervals_per_sample.csv

